# Disclaimer

These are tools developed to help me determine when my intellectual property is posted without my permission online. 
These are tools created to help me ensure that content uploaded by students to the site do not infringe my intellectual property rights.

They simply automate the process of searching the site and thus make it easier to determine which items infringe rights and which do not.

They are extremely rough and probably not very user friendly. That said, I am making them public in case others find them useful.  

These tools are provided in the hope that they will be useful to those seeking to protect their intellectual property, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

The tools are provided here with no warranty or claim that their use is consistent with any law, policy, terms of use policy, or any other constraint. IF YOU CHOOSE TO USE THEM, YOU AGREE TO REVIEW AND COMPLY WITH ALL RELEVANT POLICIES, INCLUDING SITE TERMS OF SERVICE, AND YOU AGREE THAT YOUR USE IS SOLELY AT YOUR OWN RISK.

Questions, requests, complaints, or other feedback (or pull-requests!) are welcome through this project's github page
        
        https://github.com/HatakoHaterson/CourseZero

Please understand that this is not an active project for me; I will monitor the project on github very infrequently.

-- 復讐者


View a static image of the main notebook (does not run) on Jupyter: 
    
    https://nbviewer.jupyter.org/github/HatakoHaterson/CourseZero/blob/master/Notebooks/Gather%20information.ipynb?flush_cache=true

Or run it on your own mini-server using Binder

    https://mybinder.org/v2/gh/HatakoHaterson/CourseZero/master?filepath=Notebooks%2FGather%20information.ipynb


# Get started

Once the notebook has opened in binder, a toolbar will appear at the top of the notebook containing several buttons. The 'run' button is the most important.

After you've opened the notebook in binder, click the run button above a few times. Each click runs the code in the highlighted cell. You can also press shift+return to run it. You can change which cell is highlighted with the up and down arrows or by just clicking on it.

Try it with the next cell:

In [1]:
print("You've successfully run this cell!")

You've successfully run this cell!


Since these are rough-and-ready tools, I've left in much of the code. If you want to follow along, I've left comments to myself which identify the various steps. These are either single lines which start with a '#', like this: 
    
    # Query the server for records
    code code code
    
Or, multiple lines enclosed with three double quotes: 
    
    """blah blah
    tacos are good
    """  

You'll be able to tell that a cell is running by an asterisk appearing in the brackets to the left ( [ * ]). Some tasks may take a couple of minutes to run, so it may appear frozen.

I haven't set it up with an easy way to reset things. If you want to search a different campus or different set of departments, use the 'restart kernel' button above (to the right of the run button).

The next cell will grab and import all the code that operates behind the scenes. (If you're curious, you can check it out here: https://github.com/HatakoHaterson/CourseZero)

In [2]:
# for binder
# !pip install CourseZero

# for local testing
%cd ../

import json
import pandas as pd

import requests
from bs4 import BeautifulSoup

import environment as env
from CourseZero.FileSystemTools import getSystemRoot

from IPython.display import display
from ipywidgets import widgets

from CourseZero.RequestTools import get_docs_for_campuses, get_docs_for_campus
from CourseZero.Store import DataStore
from CourseZero.Widgets.CampusAndProfIdentifiers import make_campus_selector
from CourseZero.FileSystemTools import getSystemRoot
from CourseZero.DataStorageTools import load_campus_id_data

from CourseZero.Errors import UnsetValue

/Users/adam/Dropbox/CourseZero


# Select your campus


Running the next cell will display a selection menu. Use it to select the CSU you want to search.

NB, for now this will only work with CSU campuses

In [7]:
# Load the campus ids from a file
campus_ids = load_campus_id_data(env.DEFAULT_CSU_ID_FILE)
csu_names = [c['name'] for c in campus_ids]
# Display the selector
make_campus_selector(DataStore, campus_ids)

Dropdown(description='Campus', layout=Layout(width='80%'), options=('California State University Bakersfield',…

In [8]:
# add check to make sure they actually selected something
def check_selected(data_store):
    if data_store.campus_id is None:
        print("Please select a campus to search using the dropdown menu above")
check_selected(DataStore)

# Search for courses on the selected campus

The next couple of cells will actually search for courses at the selected campus.

NB, when the search starts it will display a message like "Searching for California State University Dominguez hills". You may need to wait a few seconds for it to complete.

In [10]:
# def parse_counts(row):
#     """The server returns a field with a list of 
#     the various kinds of documents that have been uploaded.
#     This parses those counts out and adds them as columns to the frame
#     """
#     for c in row['doc_counts']:
#         row[c['category']] = c['count']
#     return row
    
# def parse_json_into_df(json_data):
#     """Parse json data into a usable dataframe
#     For now: Control which columns are kept by commenting out in to_keep
#     """
# #     to_keep = [
# # #         '@timestamp', 
# #      'canonical_course_id', 
# # #         'country', 
# #         'course_id',
# #        'course_info', 
# #         'course_name', 
# #         'course_num', 
# # #         'course_pk', 
# #         'dept_acro',
# #        'dept_id', 
# #         'dept_name', 
# #         'doc_counts', # if this isn't removed, can't dedupe
# # #         'location', 
# #         'prof_id',
# #        'prof_name', 
# # #         'school_aliases', 
# # #         'school_id', 
# #         'school_name',
# # #        'subdivision', 
# #         'term', 
# #         'title', 
# #         'total_doc_count', 
# #         'type', 
# #         'url',
# #        'verified', 
# #         'year'
# #     ]

#     data = []

#     for r in json_data:
#         try:
#             if r['total'] >0: 
#                 for row in r['results']:
#                     data.append(row)

#         except Exception as e:
#             pass
    
#     data = pd.DataFrame(data)
#     # Filter out unneeded columns
#     data = data[env.JSON_FIELDS]
#     # Add the counts of each kind of document to the frame as a column
#     data = data.apply(lambda r: parse_counts(r), axis=1)
#     # Drop the counts column so can de-dupe (list isn't hashable)
#     data.drop(['doc_counts'], axis=1, inplace=True)
#     data.drop_duplicates(inplace=True)
#     return data

# # Data frame operations
# def get_departments(frame):
#     """Extract list of departments from the results frame"""
#     depts = list(set(frame.dept_acro.tolist()))
#     depts.sort()
#     return depts

# def normalize_prof_name(prof_name):
#     return prof_name.strip().upper()

# # Frame filtration operations
# def filter_by_dept_abbrevs(frame):
#     """Uses the list of departments defined by the user to
#     return those departments from the results frame"""
#     return frame[frame['dept_acro'].isin(DataStore.departments)]

# def get_by_course_id(frame, course_ids):
#     """Finds and reeturns a course or list of courses from the results
#     frame"""
#     if type(course_ids) is not list:
#         course_ids = list(course_ids)
#     return frame[frame['course_id'].isin(course_ids)]

from CourseZero.DataHandlingTools import parse_json_into_df,get_departments

# Query for results and parse them into a dataframe
try:
    json_data = get_docs_for_campus(DataStore.campus_id, DataStore.campus_name)
    data = parse_json_into_df(json_data)
    print("{} courses identified at {}".format(len(data), DataStore.campus_name))
    depts = get_departments(data)
except UnsetValue as e:
    print(e.message)

Searching for <property object at 0x11c69b098>
162 courses identified at <property object at 0x11c69b098>


# Select department names / abbreviations

The site does not seem to standardize the departments students enter. Thus your class may be identified in some rather odd ways.

Run the next cell and select all the possible departments using the buttons which display.

In [11]:
# def make_dept_b(dept):
#     b = widgets.Button(description=dept, button_style='primary')
#     def handle(event):
#         if dept in DataStore.departments:
#             DataStore.remove_department(dept)
#             b.button_style='primary'
#         else:
#             DataStore.add_department(dept)
#             b.button_style='success'
#     b.on_click(handle)
#     return b


# def split_list(a_list):
#     half = len(a_list)//2
#     return a_list[:half], a_list[half:]

# def dept_selection(depts):
#     """Create and display the buttons for selecting which departments
#     to query"""
#     buttons = []
#     for dept in depts:
#         buttons.append(make_dept_b(dept))

#     b1, b3 = split_list(buttons)
#     b1, b2 = split_list(b1)
#     b3, b4 = split_list(b3)
    
#     display(widgets.HBox([widgets.VBox(b1), widgets.VBox(b2), widgets.VBox(b3), widgets.VBox(b4)]))

from CourseZero.Inputs import dept_selection
    
# Create the department selection buttons
dept_selection(depts)

# Display all courses with uploaded materials for the selected departments

Run the next cell and select any of the courses which look like they may be yours

In [12]:
# def make_selection_text(row):
#     """Creates the text displayed in the course selection button"""
#     t = "{prof_name} ---- {dept_acro} {course_num} ----  {course_name} ----  {course_info}"
#     return t.format(**row.to_dict())

# def make_course_b(row):
#     """Creates a button for the course defined in the row. 
#     Sets a handler on the button to toggle whether the course is selected
#     in the data store
#     """
#     layout = widgets.Layout(width='90%')
#     b = widgets.Button(description=make_selection_text(row), button_style='primary', layout=layout)
    
#     def handle(event):
#         if row['course_id'] in DataStore.course_ids:
#             DataStore.remove_course(row['course_id'])
#             b.button_style='primary'
#         else:
#             DataStore.add_course(row['course_id'])
#             b.button_style='success'
#     b.on_click(handle)
#     return b

from CourseHero.Inputs import make_course_b
# Get the selected departments
filtered = filter_by_dept_abbrevs(data)
# Make the buttons
course_buttons =[]
for i, r in filtered.iterrows():
    course_buttons.append(make_course_b(r))

display(widgets.VBox(course_buttons))

# Display material uploaded for selected courses

The following cells will query the site and retrieve the url's for all files that students have uploaded for the courses you've selected.

You will need the file url's to request that the site remove the content via this website: https://www.coursehero.com/copyright-infringement/

Remember that the url's this tool returns are simply all the files uploaded for the selected courses. 

*You must review the url's to determine that they infringe your rights.* 

Students may have uploaded their own intellectual property related to your course. You will infringe their rights if you request that their intellectual property be removed.

In [13]:
from CourseZero.RequestTools import get_file_links_from_course_page
# CHBASE = "https://www.coursehero.com{}"


# def get_file_links_from_course_page(course_page_url):
#     """Retrieves the page that would be displayed if you went to the
#     relevant page on the site, then filters out all the links to files
#     since this is what you need to file the request
#     """
#     files = []

#     html_content = get_content(env.CH_BASE_URL.format(course_page_url))
#     soup = BeautifulSoup(html_content, 'html.parser')

#     for j in soup.findAll('a'):
#         try:
#             dest = j['href']
#             fnd = "/file/"
#             if dest[: len(fnd)] == fnd:
#                 files.append(CHBASE.format(dest))
#         except Exception as e:
#             pass

#     files = list(set(files))
#     return files

def show_selected(frame):
    """Handles displaying the urls and information that has been retrieved"""
    selected = get_by_course_id(frame, DataStore.course_ids)
    for i, r in selected.iterrows():
        files = get_file_links_from_course_page(r['url'])
        t = "{prof_name} ---- {dept_acro} {course_num} ----  {course_name} ----  {course_info}"
        print(t.format(**r.to_dict()))
        for f in files:
            print(f)

show_selected(data)

Bryan Tomlin ---- ECONOMICS 310 ----  Intermediate Microeconomics ----  ECONOMICS 310 Intermediate Microeconomics
https://www.coursehero.com/file/22606123/Math-Practice-Derivatives1/
https://www.coursehero.com/file/22606574/Econ-310-Quiz-1-Spring-2017/
https://www.coursehero.com/file/22606217/Econ-310-Midterm-Review-List/
https://www.coursehero.com/file/22606206/Practice-Monopoly-and-Cournot-Duopoly-Questions-Solutions/
https://www.coursehero.com/file/22606223/Production-and-Cost/
https://www.coursehero.com/file/22606218/Choice-Under-Uncertainty-Solutions/
https://www.coursehero.com/file/22606127/Preferences-and-Indifference-Curves-Solutions/
https://www.coursehero.com/file/22606124/Math-Practice-Exponents/
https://www.coursehero.com/file/22606227/Production-and-Cost-Solutions/
https://www.coursehero.com/file/22606200/Price-Discrimination-Practice-Questions-and-Solutions/
https://www.coursehero.com/file/22606130/Utility-Practice-Problems-with-Solutions/
https://www.coursehero.com/file/

# Request takedown

CourseHero requests that you use their online form for submitting takedown requests. The form is https://www.coursehero.com/copyright-infringement/

However, they also provide the alternative of emailing or faxing a letter to DMCA@coursehero.com

You can generate the text of an email below.

    To file a DMCA takedown request, please complete the Copyright Infringement Notification form below.

    You will need to identify the infringing work(s) by providing the specific identification and location of the infringing work(s), including its URL or URLs on Course Hero. Each URL should be in the following format: https://www.coursehero.com/file/xxxx/...

    Once you submit a complete takedown request using our Copyright Infringement Notification form, our DMCA Compliance Team will quickly take action to remove the infringing content. You will be notified once the content has been removed.

    We are dedicated to helping copyright holders protect their work. As an an alternative to completing this form, you can email or fax your DMCA notification to DMCA@coursehero.com. If you have any questions or need assistance in completing your takedown request, please contact us at DMCA@coursehero.com.


In [3]:
from CourseZero.Store import TakedownStore
from CourseZero.Inputs import make_link, make_infringement_b, get_urls

# def make_link(url):
#     return "<a href='{}' target='_blank'>{}</a>".format(url,url)

# def make_infringement_b(doc, store):
#     """Creates a button for the document. 
#     Sets a handler on the button to toggle whether the doc is selected
#     in the data store
#     """
#     layout = widgets.Layout()
#     b = widgets.Button(description='NOT Infringing', button_style='primary', layout=layout)
    
#     def handle(event):
#         if doc in store.infringing_docs:
#             store.remove_doc(doc)
#             b.button_style='primary'
#             b.description='NOT Infringing'
#         else:
#             store.add_doc(doc)
#             b.button_style='success'
#             b.description='Infringing'
#     b.on_click(handle)
#     return b

# def get_urls(frame):
#     """Handles displaying the urls and information that has been retrieved"""
#     selected = get_by_course_id(frame, DataStore.course_ids)
#     for i, r in selected.iterrows():
#         files = get_file_links_from_course_page(r['url'])
#     return files

In [19]:
# Testing
docs = ['url1', 'url2', 'url3']

# For reals
# docs = get_urls(data)

#https://www.coursehero.com/copyright-infringement

Review each document below by clicking on the url (it should open the document on the CourseHero website in a new window/tab). If you believe it infringes your copyright, click the corresponding button on the right.

In [30]:
rows = [widgets.HBox([make_infringement_b(d, TakedownStore), widgets.HTML(value=make_link(d))]) for d in docs]
display(widgets.VBox(rows))

## Help with submitting through the webform

If you would like to submit the request using CourseHero's webform, here are the document names (as submitted by the student) and the corresponding url's

In [28]:
# display non-link formatted urls and doc names

## Help with email request
If you would like to make the request via email, input your information below to generate draft email text.

In [20]:

def make_text_input( input_dict ):
    """Creates a text input field. The given dictionary should have keys 'label' and 'handler'"""
    text = widgets.Text( description=input_dict[ 'label' ], name=input_dict['prop'] )
    display( text )
    text.observe( TakedownStore.event_handler )
    return text


In [23]:
[make_text_input(r) for r in TakedownStore.input_fields]

Text(value='', description='Your full name')

Text(value='', description='Your institution')

Text(value='', description='Department')

Text(value='', description='Street address')

Text(value='', description='City')

Text(value='', description='Zipcode')

Text(value='', description='Email address')

[Text(value='', description='Your full name'),
 Text(value='', description='Your institution'),
 Text(value='', description='Department'),
 Text(value='', description='Street address'),
 Text(value='', description='City'),
 Text(value='', description='Zipcode'),
 Text(value='', description='Email address')]

x
xz
xzz
s
s
a
s
d
f


In [29]:
with open('{}/takedown_request.html'.format(env.TEMPLATE_FOLDER), 'r') as o:
    template = o.read()
display(widgets.HTML(value=template.format(**TakedownStore.format_args())))

HTML(value='<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n</head>\n<body>\n\n<h1>Copy…